# WhichShroom
Here, a simple neural network using Keras is used to get accurate prediction weights to be used in a Javascript web application for predicting mushroom toxicity.

## Imports

In [1]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd

from keras.layers import Activation, Dense
from keras.models import Sequential

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Data Survey

In [2]:
df = pd.read_csv("data/MushroomClassification/mushrooms.csv")

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
list(df)

['class',
 'cap-shape',
 'cap-surface',
 'cap-color',
 'bruises',
 'odor',
 'gill-attachment',
 'gill-spacing',
 'gill-size',
 'gill-color',
 'stalk-shape',
 'stalk-root',
 'stalk-surface-above-ring',
 'stalk-surface-below-ring',
 'stalk-color-above-ring',
 'stalk-color-below-ring',
 'veil-type',
 'veil-color',
 'ring-number',
 'ring-type',
 'spore-print-color',
 'population',
 'habitat']

While I originally trained this network on all 22 of these features, I saw two factors that directed me to shorten the feature list:
* The network was training to 100% accuracy in very few epochs.
* The number of features would be cumbersome for users to input.

So, the following steps work to shorten the feature list, to capitalize on the degree of accuracy and to make a more friendly client-side.

The following was review for all 22 features to determine those with the most substantial impact on classification. However, I have only included two here as examples in order to facilitate readability.

In [4]:
print("Cap Shape")
display(pd.DataFrame({
    "Edible": df.loc[df["class"] == "e", "gill-attachment"].value_counts(),
    "Poison": df.loc[df["class"] == "p", "gill-attachment"].value_counts()
}))

print("Cap Surface")
display(pd.DataFrame({
    "Edible": df.loc[df["class"] == "e", "gill-spacing"].value_counts(),
    "Poison": df.loc[df["class"] == "p", "gill-spacing"].value_counts()
}))

Cap Shape


,Edible,Poison
f,4016,3898
a,192,18


Cap Surface


,Edible,Poison
c,3008,3804
w,1200,112


Features of import:
* cap-shape
* cap-surface
* cap-color
* bruises
* gill-spacing
* gill-size
* stalk-root
* ring-number
* ring-type
* spore-print-color
* population
* habitat

However, my intention is to select an even smaller set of features, ones that would be easy for end users to select for using the network's results. You'll thus see this list shortened further in the preprocessing below

## Preprocessing

In [5]:
labels = np.array([0 if x=="p" else 1 for x in df["class"]])

print(labels[:50])

[0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1
 0 1 1 1 1 1 0 1 1 1 1 1 1]


In [6]:
prelim_features = df.copy()
prelim_features = prelim_features.drop(["class", "stalk-root", "spore-print-color", "cap-surface",
                                        "ring-number", "ring-type", "gill-spacing", "gill-size",
                                        'odor', 'gill-attachment', 'gill-color', 'stalk-shape',
                                        'stalk-surface-above-ring', 'stalk-surface-below-ring',
                                        'stalk-color-above-ring', 'stalk-color-below-ring',
                                        'veil-type', 'veil-color'], axis=1)
prelim_features = pd.get_dummies(prelim_features, prefix=list(prelim_features), columns=list(prelim_features))

features = np.array(prelim_features.values)

print(features[:2])
print(features.shape)

[[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0]]
(8124, 31)


In [7]:
list(prelim_features)

['cap-shape_b',
 'cap-shape_c',
 'cap-shape_f',
 'cap-shape_k',
 'cap-shape_s',
 'cap-shape_x',
 'cap-color_b',
 'cap-color_c',
 'cap-color_e',
 'cap-color_g',
 'cap-color_n',
 'cap-color_p',
 'cap-color_r',
 'cap-color_u',
 'cap-color_w',
 'cap-color_y',
 'bruises_f',
 'bruises_t',
 'population_a',
 'population_c',
 'population_n',
 'population_s',
 'population_v',
 'population_y',
 'habitat_d',
 'habitat_g',
 'habitat_l',
 'habitat_m',
 'habitat_p',
 'habitat_u',
 'habitat_w']

In [8]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.2)

## Hyperparameters

In [9]:
epochs = np.array(350)
batch_size = np.array(256)

## Model

### Keras

In [10]:
model = Sequential()

model.add(Dense(8, input_dim=features.shape[1]))
model.add(Activation("sigmoid"))
model.add(Dense(1))

In [11]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

## Training

In [12]:
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=0)

In [13]:
scores = model.evaluate(test_x, test_y)

1625/1625 [==============================] - 0s 38us/step


In [14]:
print("%s: %.2f" % (model.metrics_names[1], scores[1]*100))

acc: 98.03


So, reducing the model to (effectively) five features only reduced its accuracy by ~3%. For the degree of usability this change adds, the reduction seems worthwhile.

In [15]:
model.save("MushroomClassification.h5")
with open("MushroomClassification.json", "w") as file:
    file.write(model.to_json())

In [16]:
final_weights = model.get_weights()[0]
final_biases = model.get_weights()[1]

np.savetxt("MushroomClassification_Weights.csv", final_weights, delimiter=",")
np.savetxt("MushroomClassification_Biases.csv", final_biases, delimiter=",")